# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [2]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [3]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

print(np.linalg.matrix_rank(a),'\n', a)

6 
 [[3.         3.         3.         3.         3.         3.        ]
 [3.         1.875      1.36363636 1.07142857 0.88235294 0.75      ]
 [3.         1.36363636 0.88235294 0.65217391 0.51724138 0.42857143]
 [3.         1.07142857 0.65217391 0.46875    0.36585366 0.3       ]
 [3.         0.88235294 0.51724138 0.36585366 0.28301887 0.23076923]
 [3.         0.75       0.42857143 0.3        0.23076923 0.1875    ]]


In [4]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [5]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [6]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [7]:
np.linalg.matrix_rank(a1)

6

In [8]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [11]:
# ... ENTER YOUR CODE HERE ...

def check(A):
    A = np.array(A)
    for i in range(A.shape[0]):
        if np.linalg.det(A[0:i,0:i]) == 0:
            return f'not ok, {i}_th leading minor failed'        #смотрю квадратные слайсы матрицы, начиная просто с элемента [0,0]
    return 'ok'

print('Для матрицы a: ', check(a))
print('Для матрицы a1: ', check(a1))

Для матрицы a:  ok
Для матрицы a1:  not ok, 2_th leading minor failed


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [13]:
# ... ENTER YOUR CODE HERE ...

def diy_lu_mod(A): #modified
    N = A.shape[0]
    L = np.eye(N)
    U = np.copy(A)
    p = np.arange(N) #permutations, это будет также часть вывода, чтобы восстанавливать матрицы, хранит перестановки строк
    for j in range(N-1):
        lam = np.eye(N)
        if U[j,j] == 0:                            #часть кода скопирована, добавлена только проверка неравенства нулю элемента
            for i in range(j+1, N):
                if U[i, j] != 0:
                    U[[j, i], :] = U[[i, j], :]    #переставили нужные строчки, запомнили в виде вектора, который изначально был упорядоченным от 0 до N-1
                    p[j] = i
                    break
        
        gamma = U[j+1:, j] / U[j, j]
        lam[j+1:, j] = -gamma
        U = lam @ U

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, U, p

L1, U1, p1 = diy_lu_mod(a)

print('Matrix L1: \n', L1, '\n\n Matrix U1: \n', U1, '\n\n L1 @ U1:')
print(L1 @ U1)       #quick sanity check? :)
print('\n Initial Matrix a: \n', a, '\n\n\n\n -------------------------------------------------')


#Остается научиться восстанавливать матрицу, все-таки с а нам повезло и перестановок там не было

def reconstruct(L, U, p):
    N = p.shape[0]
    A = L @ U
    for i in range(N):
        if p[i] != i:
            A[[i, p[i]], :] = A[[p[i], i], :] #меняю строчки, где необходимо
    return(A)

L2, U2, p2 = diy_lu_mod(a1)
print('Matrix L2: \n', L2, '\n\n Matrix U2: \n', U2, '\n\n L2 @ U2:')
print(L2 @ U2)
print('\n Initial Matrix a1: \n', a1)
print('\n Properly permutated L2 @ U2 \n', reconstruct(L2, U2, p2)) 

Matrix L1: 
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

 Matrix U1: 
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

 L1 @ U1:
[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]

 Initial Matrix a: 
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875